In [1]:
from pysat.formula import *
from pysat.card import *
from pysat.solvers import Solver
import numpy as np

In [2]:
shape = np.array([[0, 0, 1, 1, 0, 0],
         [0, 1, 1, 1, 1, 0],
         [0, 1, 1, 1, 1, 0],
         [0, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 1, 0]])
n = shape.shape[0]
m = shape.shape[1]

In [3]:
print(n, m)

5 6


In [4]:
cnf = CardEnc.atleast(lits=[1, 2, 3], bound=2, encoding=EncType.pairwise)
print(cnf.clauses)

[[1, 2], [1, 3], [2, 3]]


In [5]:
s = Solver(name='mc')
for clause in cnf.clauses:
    s.add_clause(clause)
s.solve()
s.get_model()

[1, 2, -3]

In [6]:
Formula.set_context("test")
test = (Atom('a') & Atom('b')) ^ Atom('c')
test.clausify()
print(test.clauses)
test = test & Atom('d')
test.clausify()
print(test.clauses)
Formula.set_context("default")

[[-3, -4], [3, 4]]
[[5], [6]]


In [7]:
with Solver(bootstrap_with=And(Atom(1), Atom(2))) as solver:
        if solver.solve():
            print("satisfiable")
            res = solver.get_model()
        else:
            print("unsatisfiable")

satisfiable


In [23]:
def rectangle_formula(n: int, m: int, k: int, vpool: IDPool, solver: Solver):
    t = lambda k, i: Atom(vpool.id('t{},{}'.format(k, i)))
    b = lambda k, i: Atom(vpool.id('b{},{}'.format(k, i)))
    l = lambda k, j: Atom(vpool.id('l{},{}'.format(k, j)))
    r = lambda k, j: Atom(vpool.id('r{},{}'.format(k, j)))

    constraints = []
    for i in 
            

    return constraints
    #rect_constr = And(*[constraint for constraint in constraints])
    #for clause in rect_constr:
    #    solver.add_clause(clause)

In [21]:
from pysat.formula import *
from pysat.card import *
from pysat.solvers import Solver
import numpy as np

# Definim la forma que volem cobrir amb un rectangle
shape = np.array([[0, 0, 0, 1, 1, 0],
                  [0, 1, 1, 1, 1, 0],
                  [1, 1, 1, 1, 1, 0],
                  [0, 1, 1, 1, 1, 1],
                  [0, 0, 1, 1, 1, 0]])
n = shape.shape[0]
m = shape.shape[1]

# De moment només treballarem amb k=1 rectangles
vpool = IDPool()  # IDPool() controla el canvi de nom <-> literal (nombre >= 1)
e = lambda i, j: Atom(vpool.id('e{},{}'.format(i, j)))
s = lambda i, j: Atom(vpool.id('s{},{}'.format(i, j)))
l = lambda i: Atom(vpool.id('l{}'.format(i)))
r = lambda i: Atom(vpool.id('r{}'.format(i)))
t = lambda j: Atom(vpool.id('t{}'.format(j)))
c = lambda j: Atom(vpool.id('c{}'.format(j)))

# Definim totes les fórmules que després hauran d'anar juntes amb un And
constraints = []

# Restriccions files
constraints.append(Equals(l(0), r(0)))
for i in range(1, n):
    constraints.append(l(i) >> And(r(i), ~r(i-1)))  # >> vol dir implicació
    constraints.append(And(r(i), ~l(i)) >> r(i-1))

# Restriccions columnes
constraints.append(Equals(t(0), c(0)))
for j in range(1, m):
    constraints.append(t(j) >> And(c(j), ~c(j-1)))
    constraints.append(And(c(j), ~t(j)) >> c(j-1))

for i in range(n):
    for j in range(m):
        # Variable de seleccionar
        constraints.append(Equals(s(i,j), And(r(i), c(j))))
        if shape[i][j] != 0:
            # Si seleccionem la cel·la però no estava en la regió original compta com un error
            constraints.append(Equals(e(i,j), ~s(i,j)))
        else:
            # Si no seleccionem la cel·la que estava en la regió original compta com un error
            constraints.append(Equals(e(i,j), s(i,j)))
        
formula = And(*[constraint for constraint in constraints])

satisfiable = True
bound = n*m
last_model = None
while satisfiable:
    with Solver(name='gc4', bootstrap_with=formula) as solver:
        solver.add_atmost(lits=[l(i).name for i in range(n)], k=1)  # AMO(li)
        solver.add_atmost(lits=[t(j).name for j in range(m)], k=1)  # AMO(tj)
        solver.add_atmost(lits=[e(i,j).name for i in range(n) for j in range(m)], k=bound)
        if solver.solve():
            print("satisfiable")
            last_model = solver.get_model()
            new_bound = 0
            for i in range(n):
                for j in range(m):
                    if e(i,j).name in solver.get_model():
                        print(e(i,j).name)
                        new_bound += 1
            print("New bound", new_bound)
            bound = new_bound - 1
        else:
            satisfiable = False
            print("unsatisfiable")


result = np.zeros((n,m))
for i in range(n):
    for j in range(m):
        if s(i,j).name in last_model:
            result[i][j] = 1

print(result)

satisfiable
30
32
38
40
42
44
48
50
52
54
56
62
64
66
68
70
76
78
80
New bound 19
satisfiable
32
38
40
42
44
48
50
52
54
56
62
64
66
68
70
76
78
80
New bound 18
satisfiable
38
40
42
44
48
50
52
54
56
62
64
66
68
70
76
78
80
New bound 17
satisfiable
30
32
44
48
50
52
54
56
62
64
66
68
70
76
78
80
New bound 16
satisfiable
26
28
48
50
52
54
56
62
64
66
68
70
76
78
80
New bound 15
satisfiable
30
32
36
44
56
62
64
66
68
70
76
78
80
New bound 13
satisfiable
28
38
48
50
62
64
66
68
70
76
78
80
New bound 12
satisfiable
26
28
48
62
64
66
68
70
76
78
80
New bound 11
satisfiable
30
32
44
48
56
68
70
76
78
80
New bound 10
satisfiable
30
32
36
60
70
76
78
80
New bound 8
satisfiable
30
32
48
70
76
78
80
New bound 7
satisfiable
30
32
48
70
74
New bound 5
unsatisfiable
[[0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 0.]]


In [13]:
print(last_model)

[-1, -2, 3, 4, -5, 6, -7, 8, -9, -10, -11, -12, 13, 14, -15, 16, -17, 18, -19, 20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, 37, -38, 39, -40, 41, -42, 43, -44, -45, -46, -47, -48, 49, -50, 51, -52, 53, -54, 55, -56, -57, -58, -59, -60, 61, -62, 63, -64, 65, -66, 67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79, -80, -81, -82, 83, 84, 85, -86, -87, -88, 89, 90, -91, -92, 93, 94, -95, -96, 97, -98, -99, 100, 101, 102, -103, -104, -105, 106, 107, -108, -109, 110, 111, -112, -113, 114, 115, -116, -117, 118, -119, -120, -121, 122, 123, -124, 125, 126, -127, 128, 129, -130, 131, 132, -133, 134, 135, -136, 137, 138, -139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, -154, 155, 156, -157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, -172, 173, 174, -175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, -190, 191, 192, -193, 194, 195, -196, 197, 198, -199, 200, 201, -202, 

In [8]:
from pysat.formula import *
from pysat.card import *
from pysat.solvers import Solver
import numpy as np

# Definim la forma que volem cobrir amb un rectangle
shape = np.array([[0, 0, 0, 1, 1, 0],
                  [0, 1, 1, 1, 1, 0],
                  [1, 1, 0, 0, 0, 0],
                  [0, 1, 1, 1, 1, 1],
                  [0, 0, 1, 1, 1, 1]])
#shape = np.array([[1,1,1,0],[1,1,0,0],[1,0,0,0]])
n = shape.shape[0]
m = shape.shape[1]

# De moment només treballarem amb k=1 rectangles
vpool = IDPool()  # IDPool() controla el canvi de nom <-> literal (nombre >= 1)
e = lambda i, j: Atom(vpool.id('e{},{}'.format(i, j)))
s = lambda k, i, j: Atom(vpool.id('s{},{},{}'.format(k, i, j)))
l = lambda k, i: Atom(vpool.id('l{},{}'.format(k, i)))
r = lambda k, i: Atom(vpool.id('r{},{}'.format(k, i)))
t = lambda k, j: Atom(vpool.id('t{},{}'.format(k, j)))
c = lambda k, j: Atom(vpool.id('c{},{}'.format(k, j)))

# Definim totes les fórmules que després hauran d'anar juntes amb un And
k_rect = 3
constraints = [[] for k in range(k_rect)]
for k in range(k_rect):
    # Restriccions files
    constraints[k].append(Equals(l(k,0), r(k,0)))
    for i in range(1, n):
        constraints[k].append(l(k,i) >> And(r(k,i), ~r(k,i-1)))  # >> vol dir implicació
        constraints[k].append(And(r(k,i), ~l(k,i)) >> r(k,i-1))
    
    # Restriccions columnes
    constraints.append(Equals(t(k,0), c(k,0)))
    for j in range(1, m):
        constraints.append(t(k,j) >> And(c(k,j), ~c(k,j-1)))
        constraints.append(And(c(k,j), ~t(k,j)) >> c(k,j-1))

err_constraints = []
for i in range(n):
    for j in range(m):
        # Variable de seleccionar
        for k in range(k_rect):
            constraints[k].append(Equals(s(k,i,j), And(r(k,i), c(k,j))))
        if shape[i][j] != 0:
            # Si seleccionem la cel·la però no estava en la regió original compta com un error
            err_constraints.append(Equals(e(i,j), ~Or(*[s(k,i,j) for k in range(k_rect)])))
        else:
            # Si no seleccionem la cel·la que estava en la regió original compta com un error
            err_constraints.append(Equals(e(i,j), Or(*[s(k,i,j) for k in range(k_rect)])))

formula = And(*[c for c in err_constraints], *[And(*[c for c in constraints[k]]) for k in range(k_rect)])

satisfiable = True
bound = n*m
last_model = None
while satisfiable:
    with Solver(name='gc4', bootstrap_with=formula) as solver:
        if not solver.solve():
            print("unsatisfiability confirmed")
            break
        for k in range(k_rect):
            solver.add_atmost(lits=[l(k,i).name for i in range(n)], k=1)  # AMO(li)
            solver.add_atmost(lits=[t(k,j).name for j in range(m)], k=1)  # AMO(tj)
        solver.add_atmost(lits=[e(i,j).name for i in range(n) for j in range(m)], k=bound)
        if solver.solve():
            print("satisfiable")
            last_model = solver.get_model()
            new_bound = 0
            for i in range(n):
                for j in range(m):
                    if e(i,j).name in solver.get_model():
                        new_bound += 1
            print("New bound", new_bound)
            bound = new_bound - 1
        else:
            satisfiable = False
            print("unsatisfiable")

result = np.zeros((n,m))
rectangles = [np.zeros((n,m)) for k in range(k_rect)]
if last_model:
    for i in range(n):
        for j in range(m):
            for k in range(k_rect):
                if s(k,i,j).name in last_model:
                    result[i][j] = 1
                    rectangles[k][i][j] = 1

print(result)
for k in range(k_rect):
    print("Rectangle", k)
    print(rectangles[k])

satisfiable
New bound 17
satisfiable
New bound 16
satisfiable
New bound 15
satisfiable
New bound 14
satisfiable
New bound 13
satisfiable
New bound 12
satisfiable
New bound 11
satisfiable
New bound 10
satisfiable
New bound 9
satisfiable
New bound 8
satisfiable
New bound 7
satisfiable
New bound 6
satisfiable
New bound 5
satisfiable
New bound 4
satisfiable
New bound 3
satisfiable
New bound 2
unsatisfiable
[[0. 0. 0. 1. 1. 0.]
 [0. 1. 0. 1. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1.]]
Rectangle 0
[[0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
Rectangle 1
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1.]]
Rectangle 2
[[0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [ ]:
def rectangle_formula(n: int, m: int, k: int, vpool: IDPool, solver: Solver):
    t = lambda k, i: Atom(vpool.id('t{},{}'.format(k, i)))
    b = lambda k, i: Atom(vpool.id('b{},{}'.format(k, i)))
    l = lambda k, j: Atom(vpool.id('l{},{}'.format(k, j)))
    r = lambda k, j: Atom(vpool.id('r{},{}'.format(k, j)))

    constraints = []
    constraints.append(Equals(t(k,1), Or(*[c(k,1,j) for j in range(1, m+1)])))
    for i in range(2, n+1):
        constraints.append(Equals(t(k,i), Or(*[And(Neg(c(k,i-1,j)), c(k,i,j)) for j in range(1, m+1)])))
    solver.add_atmost(lits=[-t(k,i).name for i in range(1, n+1)], k=n-1)

    constraints.append(Equals(b(k,n), Or(*[c(k,n,j) for j in range(1, m+1)])))
    for i in range(1, n):
        constraints.append(Equals(b(k,i), Or(*[And(Neg(c(k,i+1,j)), c(k,i,j)) for j in range(1, m+1)])))
    solver.add_atmost(lits=[-b(k,i).name for i in range(1, n+1)], k=n-1)

    constraints.append(Equals(l(k,1), Or(*[c(k,i,1) for i in range(1, n+1)])))
    for j in range(2, m+1):
        constraints.append(Equals(l(k,j), Or(*[And(Neg(c(k,i,j-1)), c(k,i,j)) for i in range(1, n+1)])))
    solver.add_atmost(lits=[-l(k,j).name for j in range(1, m+1)], k=m-1)

    constraints.append(Equals(r(k,m), Or(*[c(k,i,m) for i in range(1, n+1)])))
    for j in range(1, m):
        constraints.append(Equals(r(k,j), Or(*[And(Neg(c(k,i,j+1)), c(k,i,j)) for i in range(1, n+1)])))
    solver.add_atmost(lits=[-r(k,j).name for j in range(1, m+1)], k=m-1)

    for i in range(1, n+1):
        for j in range(1, m+1):
            

    return constraints
    #rect_constr = And(*[constraint for constraint in constraints])
    #for clause in rect_constr:
    #    solver.add_clause(clause)


with Solver(name='gc4') as solver:
    vpool = IDPool()
    c = lambda k, i, j: Atom(vpool.id('c{},{},{}'.format(k, i, j)))
    k_rect = 1
    constraints = []
    for k in range(1, k_rect+1):
        constraints += rectangle_formula(n, m, k, vpool, solver)
    s = lambda i, j: Atom(vpool.id('s{},{}'.format(i, j)))
    e = lambda i, j: Atom(vpool.id('e{},{}'.format(i, j)))
    for i in range(1, n+1):
        for j in range(1, m+1):
            #constraints.append(Equals(s(i,j), Or(*[c(k,i,j) for k in range(1,k_rect+1)])))
            #constraints.append(Equals(e(i,j), Neg(s(i,j)) if shape[i-1][j-1] != 0 else s(i,j)))
    
    err_constr = And(*[constraint for constraint in constraints])
    #err_constr.clausify()
    print(err_constr)
    for clause in err_constr:
        solver.add_clause(clause)
    bound = n*m
    #solver.add_atmost(lits=[e(i,k).name for i in range(1, n+1) for j in range(1, m+1)], k=bound)          
    if solver.solve():
        print("satisfiable")
        for o in solver.get_model():
            if (o > 0 and vpool.obj(o)):
                print(vpool.obj(o))
    else:
        print("unsatisfiable")